In [0]:
!pip install emcee
!pip install corner
!pip install batman-package

# Read fits, plot data, determine period, find/mask transits, (try to) determine star rotation period:

In [0]:
import numpy as np
from astropy.io import fits
import lightkurve as lk
import matplotlib.pyplot as plt
from astropy.timeseries import BoxLeastSquares
from scipy.signal import savgol_filter
import exoplanet as xo
import pymc3 as pm
import theano.tensor as tt

#261136679

ID = 'TIC146264536'
MISSION = 'TESS'

#tpf_file = lk.search_lightcurvefile("TIC36734222", mission=MISSION).download_all(download_dir=LK_CACHE_DIR)
#tpf_file = lk.search_targetpixelfile("TIC 261136679", sector = 1).download()

#wasp-122
#tpf_file = '/Users/ivshina/Desktop/tides/data/a_tess-s0007-3-1_108.301475_-42.409754_10x10_astrocut.fits'

#wasp-12
tpf_file = '/Users/ivshina/Desktop/tides/data/wasp12_6x6.fits'

#with tpf_file.hdu as hdu:
with fits.open(tpf_file, mode="readonly") as hdu: 
    tpf = hdu[1].data
    tpf_hdr = hdu[1].header

texp = tpf_hdr["FRAMETIM"] * tpf_hdr["NUM_FRM"]
texp /= 60.0 * 60.0 * 24.0
time = tpf["TIME"]
flux = tpf["FLUX"]
m = np.any(np.isfinite(flux), axis=(1, 2)) & (tpf["QUALITY"] == 0)
ref_time = 0.5 * (np.min(time[m]) + np.max(time[m]))
time = np.ascontiguousarray(time[m]) #- ref_time, dtype=np.float64)
flux = np.ascontiguousarray(flux[m], dtype=np.float64)
mean_img = np.median(flux, axis=0)
#plt.imshow(mean_img.T, cmap="gray_r")
plt.title("TESS image of WASP-12")
plt.xticks([])
plt.yticks([]);
#plt.show()

print('time shape ', time)
#################################################################
#Aperture selection

# Sort pixels  
order = np.argsort(mean_img.flatten())[::-1]
 
# Estimate the windowed scatter in a lightcurve
def estimate_scatter_with_mask(mask):
    f = np.sum(flux[:, mask], axis=-1)
    #smooth data
    smooth = savgol_filter(f, 1001, polyorder=5)
    return 1e6 * np.sqrt(np.median((f / smooth - 1) ** 2))


# Loop over pixels ordered by brightness and add them one-by-one
# to the aperture
masks, scatters = [], []
for i in range(10, 100):
    msk = np.zeros_like(mean_img, dtype=bool)
    msk[np.unravel_index(order[:i], mean_img.shape)] = True
    scatter = estimate_scatter_with_mask(msk)
    masks.append(msk)
    scatters.append(scatter)

# Choose the aperture that minimizes the scatter
pix_mask = masks[np.argmin(scatters)]
 
# Plot the selected aperture
#plt.imshow(mean_img.T, cmap="gray_r")
#plt.imshow(pix_mask.T, cmap="Reds", alpha=0.3)
plt.title("selected aperture")
plt.xticks([])
plt.yticks([]);
#plt.show()


#print('time shape', time.shape)
#print('flux', flux[:, pix_mask].shape)
plt.figure(figsize=(10, 5))
sap_flux = np.sum(flux[:, pix_mask], axis=-1)
#sap_flux = (sap_flux / np.median(sap_flux) - 1) * 1e3
plt.plot(time, sap_flux, "k")
plt.xlabel("time [days]")
plt.ylabel("relative flux [ppt]")
plt.title("raw light curve")
plt.xlim(time.min(), time.max());
plt.show()

# De-trending (systematic and random noise sources)

# Build the first order PLD basis
X_pld = np.reshape(flux[:, pix_mask], (len(flux), -1))
X_pld = X_pld / np.sum(flux[:, pix_mask], axis=-1)[:, None]

# Build the second order PLD basis and run PCA to reduce the number of dimensions
X2_pld = np.reshape(X_pld[:, None, :] * X_pld[:, :, None], (len(flux), -1))
U, _, _ = np.linalg.svd(X2_pld, full_matrices=False)
X2_pld = U[:, : X_pld.shape[1]]

# Construct the design matrix and fit for the PLD model
X_pld = np.concatenate((np.ones((len(flux), 1)), X_pld, X2_pld), axis=-1)
XTX = np.dot(X_pld.T, X_pld)
w_pld = np.linalg.solve(XTX, np.dot(X_pld.T, sap_flux))
pld_flux = np.dot(X_pld, w_pld)



##############################################
# Periodogram
model = BoxLeastSquares(time, sap_flux - pld_flux)
periodogram = model.autopower(0.2)
plt.plot(periodogram.period, periodogram.power)  
plt.xlabel("Period [day]")
plt.ylabel("Power")
plt.text(10,2117,
    "period = {0:.4f} d".format(periodogram.period[np.argmax(periodogram.power)]))
#print('Period:', periodogram.period[np.argmax(periodogram.power)])

 
period_grid = np.exp(np.linspace(np.log(0.05), np.log(15), 50000))

bls = BoxLeastSquares(time, sap_flux - pld_flux)
bls_power = bls.power(period_grid, 0.02, oversample=20)
plt.xlabel("time [days]")
plt.ylabel("de-trended flux [ppt]")

# Save the highest peak as the planet candidate
index = np.argmax(bls_power.power)
bls_period = bls_power.period[index]
bls_t0 = bls_power.transit_time[index]
print('t 0 ', bls_t0)
bls_depth = bls_power.depth[index]
transit_mask = bls.transit_mask(time, bls_period, 0.2, bls_t0)


x = np.ascontiguousarray(time, dtype=np.float64)
y = np.ascontiguousarray(sap_flux - pld_flux, dtype=np.float64) 
 
# Plot the folded transit
#ax = axes[1]
x_fold = (time - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period
m = np.abs(x_fold) < 0.4
not_transit = ~transit_mask

# folded data with transit masked:
total_mask = m & not_transit
flux_masked = sap_flux[total_mask]
time_masked = x_fold[total_mask]
times_masked = time[total_mask] # times (not relative)
# folded data with transit included:
flux_folded = sap_flux[m]
time_folded = x_fold[m]
times = time[m]

 
# save data
np.savetxt('/Users/ivshina/Desktop/time_masked.txt', time_masked)
np.savetxt('/Users/ivshina/Desktop/flux_masked.txt', flux_masked)
np.savetxt('/Users/ivshina/Desktop/time_folded.txt', time_folded)
np.savetxt('/Users/ivshina/Desktop/flux_folded.txt', flux_folded)

flux_masked = np.loadtxt('/Users/ivshina/Desktop/tides/fitting_params/flux_masked.txt')
time_masked = np.loadtxt('/Users/ivshina/Desktop/tides/fitting_params/time_masked.txt')
flux_folded = np.loadtxt('/Users/ivshina/Desktop/tides/fitting_params/flux_folded.txt')
time_folded = np.loadtxt('/Users/ivshina/Desktop/tides/fitting_params/time_folded.txt')

fits = []
transit_start_index = 0
for i in range(flux_masked.shape[0]-20):
    
    if np.abs(time_masked[i]-time_masked[i+1])>0.6:
        # folded flux with transit masked
        tr_flux = flux_masked[transit_start_index:i]
        tr_time = time_masked[transit_start_index:i]
        # linear fit
        k, b = np.polyfit(tr_time, tr_flux, deg=1)
        fits.append([k,b])
        transit_start_index = i+1

 

tr_flux = flux_masked[transit_start_index:]
tr_time = time_masked[transit_start_index:]
# linear fit
k, b = np.polyfit(tr_time, tr_flux, deg=1)
fits.append([k,b])
plt.plot(time[transit_mask], sap_flux[transit_mask], ".k")
plt.ylim(-30, 70)
#plt.xlim(-0.3, 0.3)
plt.ylabel("test de-trended flux [ppt]")
plt.show()
print('number of transits: ', len(fits))  

# Find linear fit and de-trend each transit separately

In [0]:
import numpy as np
from astropy.io import fits
import lightkurve as lk
import matplotlib.pyplot as plt
from astropy.timeseries import BoxLeastSquares
from scipy.signal import savgol_filter
import exoplanet as xo
import pymc3 as pm
import theano.tensor as tt

stds = []

for i in range(21):
    transit_masked = np.loadtxt('/Users/ivshina/Desktop/tides/wasp12_2min_cadence/transit_masked/flux/flux_%d.txt' % i)
    time_masked = np.loadtxt('/Users/ivshina/Desktop/tides/wasp12_2min_cadence/transit_masked/time/time_%d.txt' % i)
    transit = np.loadtxt('/Users/ivshina/Desktop/tides/wasp12_2min_cadence/transit/flux/flux_%d.txt' % i)
    time = np.loadtxt('/Users/ivshina/Desktop/tides/wasp12_2min_cadence/transit/time/time_%d.txt' % i)    
    # linear fit
    k, b = np.polyfit(time_masked, transit_masked, deg=1)
    fit = k*time+b
    corrected_flux = transit/fit
    fit_y = k*time_masked+b
    y = transit_masked/fit_y
    stds.append(np.std(y))
    np.savetxt('/Users/ivshina/Desktop/tides/wasp12_2min_cadence/corrected_flux/corrected_flux_%d.txt' %i, corrected_flux)

np.savetxt('/Users/ivshina/Desktop/stds.txt', np.array(stds))

After masking the transits, in the file of folded data with transits included and in the file with transits excluded, separate the data into 1 file per transit

In [0]:
import numpy as np
from astropy.io import fits
import lightkurve as lk
import matplotlib.pyplot as plt
from astropy.timeseries import BoxLeastSquares
from scipy.signal import savgol_filter
import exoplanet as xo



times = np.loadtxt('/Users/ivshina/Desktop/wasp12_2min_cadence/transit_masked/time_masked.txt')
time_folded = np.loadtxt('/Users/ivshina/Desktop/wasp12_2min_cadence/transit_masked/folded_time_masked.txt')
flux = np.loadtxt('/Users/ivshina/Desktop/wasp12_2min_cadence/transit_masked/flux_masked.txt')

indx = []
indx.append(0)
for i in range(1,time_folded.shape[0]):
    if (time_folded[i] < 0 and time_folded[i-1] > 0):
        indx.append(i)

print(indx)


for i in range(len(indx)):
    if indx[i] != 8704:
        data_ = flux[indx[i]:indx[i+1]]
        time_ = times[indx[i]:indx[i+1]]
        np.savetxt('/Users/ivshina/Desktop/wasp12_2min_cadence/transit_masked/time/time_%d.txt' %i, time_)
        np.savetxt('/Users/ivshina/Desktop/wasp12_2min_cadence/transit_masked/flux/flux_%d.txt' %i, data_)
    else:
        data_ = flux[indx[i]:]
        time_ = times[indx[i]:]
        np.savetxt('/Users/ivshina/Desktop/wasp12_2min_cadence/transit_masked/time/time_%d.txt' %i, time_)
        np.savetxt('/Users/ivshina/Desktop/wasp12_2min_cadence/transit_masked/flux/flux_%d.txt' %i, data_)

# Read 2-minute cadence TESS data and visualize it

In [0]:
import numpy as np
from astropy.io import fits
import lightkurve as lk
import matplotlib.pyplot as plt
from astropy.timeseries import BoxLeastSquares
from scipy.signal import savgol_filter
import exoplanet as xo
import pymc3 as pm
import theano.tensor as tt

fits_file = '/Users/ivshina/Desktop/tides/tess2019357164649-s0020-0000000086396382-0165-s_lc.fits'

#print(fits.info(fits_file))
print(fits.getdata(fits_file, ext=1).columns)

with fits.open(fits_file, mode="readonly") as hdulist:
    tess_bjds = hdulist[1].data['TIME']
    sap_fluxes = hdulist[1].data['SAP_FLUX']
    pdcsap_fluxes = hdulist[1].data['PDCSAP_FLUX']
    print('Header:', hdulist[0].header)




# Start figure and axis.
fig, ax = plt.subplots()

# Plot the timeseries in black circles.
ax.plot(tess_bjds, pdcsap_fluxes, '.k')
plt.xlabel('Time')
plt.ylabel('Flux')
plt.show()

with fits.open(fits_file, mode="readonly") as hdulist:
    aperture = hdulist[2].data

# Start figure and axis.
fig, ax = plt.subplots()

# Display the pixels as an image.
cax = ax.imshow(aperture, cmap=plt.cm.YlGnBu_r, origin="lower")

# Add a color bar.
cbar = fig.colorbar(cax)

# Add a title to the plot.
fig.suptitle("WASP-12b Aperture")
#plt.show()


#wasp-12
tpf_file = '/Users/ivshina/Desktop/tess2019357164649-s0020-0000000086396382-0165-s_lc.fits'
time = tess_bjds

m = np.isfinite(pdcsap_fluxes)
time = np.ascontiguousarray(time[m])
pdcsap_fluxes = np.ascontiguousarray(pdcsap_fluxes[m])
#time  = [y for x in time.tolist() for y in x]
#pdcsap_fluxes  = [y for x in pdcsap_fluxes.tolist() for y in x]
print(time.tolist()[:200])#pdcsap_fluxes[:200])

##############################################
# Periodogram
model = BoxLeastSquares(time, pdcsap_fluxes)
periodogram = model.autopower(0.2)
plt.plot(periodogram.period, periodogram.power)  
plt.xlabel("Period [day]")
plt.ylabel("Power")
plt.text(10,2117,
    "period = {0:.4f} d".format(periodogram.period[np.argmax(periodogram.power)]))
#print('Period:', periodogram.period[np.argmax(periodogram.power)])

 
####
period_grid = np.exp(np.linspace(np.log(0.05), np.log(15), 50000))

bls = BoxLeastSquares(time, pdcsap_fluxes)
bls_power = bls.power(period_grid, 0.02, oversample=20)
plt.xlabel("time [days]")
plt.ylabel("de-trended flux [ppt]")

# Save the highest peak as the planet candidate
index = np.argmax(bls_power.power)
bls_period = bls_power.period[index]
bls_t0 = bls_power.transit_time[index]
print('t 0 ', bls_t0)
bls_depth = bls_power.depth[index]
transit_mask = bls.transit_mask(time, bls_period, 0.2, bls_t0)


x = np.ascontiguousarray(time, dtype=np.float64)
y = np.ascontiguousarray(pdcsap_fluxes, dtype=np.float64) 
 
# Plot the folded transit
#ax = axes[1]
x_fold = (time - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period
m = np.abs(x_fold) < 0.4
not_transit = ~transit_mask

# folded data with transit masked:
total_mask = m & not_transit
flux_masked = pdcsap_fluxes[total_mask]
time_masked = x_fold[total_mask]
times_masked = time[total_mask] # times (not relative)
# folded data with transit included:
flux_folded = pdcsap_fluxes[m]
time_folded = x_fold[m]
times = time[m]
'''
np.savetxt('/Users/ivshina/Desktop/times.txt', times)
np.savetxt('/Users/ivshina/Desktop/flux.txt', flux_folded)
np.savetxt('/Users/ivshina/Desktop/time_folded.txt', time_folded)

# save data
np.savetxt('/Users/ivshina/Desktop/folded_time_masked.txt', time_masked)
np.savetxt('/Users/ivshina/Desktop/time_masked.txt', times_masked)
np.savetxt('/Users/ivshina/Desktop/flux_masked.txt', flux_masked)
#np.savetxt('/Users/ivshina/Desktop/time_masked.txt', time_masked)
#np.savetxt('/Users/ivshina/Desktop/flux_masked.txt', flux_masked)
#np.savetxt('/Users/ivshina/Desktop/time_folded.txt', time_folded)
#np.savetxt('/Users/ivshina/Desktop/flux_folded.txt', flux_folded)

#flux_masked = np.loadtxt('/Users/ivshina/Desktop/flux_masked.txt')
#time_masked = np.loadtxt('/Users/ivshina/Desktop/time_masked.txt')
#flux_folded = np.loadtxt('/Users/ivshina/Desktop/flux_folded.txt')
#time_folded = np.loadtxt('/Users/ivshina/Desktop/time_folded.txt')
'''

# Fit transit model, produce light curves, determine mid-transit times:

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid.anchored_artists import AnchoredText
from scipy.stats import binned_statistic
import batman
import emcee
import corner
import os, sys, time
import pandas as pd

In [0]:
# Exoplanetary system parameters
targ_name = 'WASP-43b'
per_i = 0.813475 # period (days) 

rp_i = 0.1787 # Rp/R* (planet's radius in terms of stellar radii)
a_i = 5.48727 #(semi-major axis in terms of stellar radii)
b_i = 0.5 # impact parameter

u1 = 0.5	# Limb Darkening coefficient 1
u2 = 0.1144 # Limb Darkening coefficient 2

# MCMC parameters
nsteps = 20000
burn_in = 1000
ndim = 4
nwalkers = 100

In [0]:
# Priors.
def lnprior(theta, t0_i):
	t0, rp, a, b = theta
	if (t0_i-1 < t0 < t0_i+1) \
	and (0. < rp) \
	and (b < a) \
	and (0. <= b < 1.):
		return 0
	return -np.inf


def lnlike(theta, x, y, sigma, per=per_i):
  t0, rp, a, b = theta
  # From Claret et al. 2012/13
  u1 = 0.5	# Limb Darkening coefficient 1
  u2 = 0.1144 # Limb Darkening coefficient 2
  # Set up transit parameters.
  params = batman.TransitParams()
  params.t0 = t0
  params.per = per
  params.rp = rp
  params.a = a
  params.inc = 82
  params.ecc = 0.0092
  params.w = 96
  params.u = [u1, u2]
  params.limb_dark = 'quadratic'
  # Initialize the transit model.
  m_init = batman.TransitModel(params, x)
  model = m_init.light_curve(params)  
  inv_sigma2 = 1.0 / (sigma**2)
  return -0.5*(np.sum((y-model)**2*inv_sigma2))
	
 

# Define log of probability function.
def lnprob(theta, x, y, t0_i, sigma):
  lp = lnprior(theta, t0_i)
  if not np.isfinite(lp):
    return -np.inf
  return lp + lnlike(theta, x, y, sigma)

In [0]:
t0s = []
t0s_best = []
stds = np.loadtxt('stds.txt')

for i in range(1,27):
    print('transit # ', i)
    times = np.loadtxt('time/time_%d.txt' % i)
    fluxes = np.loadtxt('corrected_flux/corrected_flux_%d.txt' % i)
    data = pd.read_csv('corrected_flux/corrected_flux_%d.txt' % i, sep=" ", header=None)
    idx = data.idxmin()
    t0_i = times[idx][0]
    print('initial t0: ', t0_i)
    sigma = stds[i]
    print(t0_i) 


    initial_params = t0_i, rp_i, a_i, b_i 

    # Initialize walkers around maximum likelihood.
    pos = [initial_params + 1e-5*np.random.randn(ndim) for i in range(nwalkers)]

    # Set up sampler.
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(np.array(times), np.array(fluxes), t0_i, sigma))

    # Run MCMC for n steps and display progress bar.
    width = 50
    for m, result in enumerate(sampler.sample(pos, iterations=nsteps)):
    	n = int((width+1) * float(m) / nsteps)
    	sys.stdout.write("\r{}[{}{}]{}".format('sampling... ', '#' * n, ' ' * (width - n), ' (%s%%)' % str(100. * float(m) / nsteps)))
    sys.stdout.write("\n")
    print ('Sampling complete!')

    samples = sampler.chain
 
    # Discard burn-in. 
    samples = samples[:, burn_in:, :].reshape((-1, ndim))

    # Final params and uncertainties based on the 16th, 50th, and 84th percentiles of the samples in the marginalized distributions.
    t0_mcmc, rp_mcmc, a_mcmc, b_mcmc = map(
	    lambda v: (v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(samples, [16, 50, 84], axis=0)))
     
    t0s.append([round(t0_mcmc[0],4),round(t0_mcmc[1],4), round(t0_mcmc[2],4)])
    np.savetxt('t0s.txt', np.array(t0s))

    samples = sampler.flatchain
    theta_max  = samples[np.argmax(sampler.flatlnprobability)]
    t0s_best.append(theta_max[0])
    np.savetxt('t0s_best.txt', np.array(t0s_best))
    print('radius: ', rp_mcmc)
    print('best radius: ', theta_max[1])
    # Plot the final transit model.
    # create array for flux uncertanties
    ferr = []
    for k in range(times.shape[0]):
	    ferr.append(stds[i])
    ferr = np.asarray(ferr)

    params_final = batman.TransitParams()
    params_final.t0 = theta_max[0]
    params_final.per = per_i
    params_final.rp = theta_max[1]
    params_final.a = theta_max[2]
    params_final.inc = np.arccos(theta_max[3] / theta_max[2]) * (180. / np.pi)
    params_final.ecc = 0.0092
    params_final.w = 96
    params_final.u = [u1, u2]
    params_final.limb_dark = "quadratic"
    tl = np.linspace(min(times),max(times),5000)
    m = batman.TransitModel(params_final, tl)
    f_final = m.light_curve(params_final)
    final_fig, ax = plt.subplots(figsize=(10,8))
    ax.set_title(targ_name)
    ax.errorbar(times,fluxes,yerr=ferr,fmt='k.',capsize=0,alpha=0.4,zorder=1)
    ax.plot(tl, f_final, 'r-',alpha=0.8,lw=3,zorder=2)
    if annotate_plot == True:
    	ant = AnchoredText('$T_0 = %s^{+%s}_{-%s}$ \n $R_p/R = %s^{+%s}_{-%s}$' % (round(t0_mcmc[0],4),round(t0_mcmc[1],4),
    	round(t0_mcmc[2],4),round(rp_mcmc[0],4),round(rp_mcmc[1],4),round(rp_mcmc[2],4)), prop=dict(size=11), frameon=True, loc=3)
    	ant.patch.set_boxstyle('round,pad=0.,rounding_size=0.2')
    	ax.add_artist(ant)
    ax.set_xlabel("Time")
    ax.set_ylabel("Relative Flux")
    ax.legend(('BATMAN','TESS'), loc=2)
     
    if save_results == True:
        save_to = 'wasp43_figures'
        final_fig.savefig('wasp43_figures/MCMCfit%d.png' %i, bbox_inches='tight')

# Fit constant period model to the data to determine orbital period

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid.anchored_artists import AnchoredText
from scipy.stats import binned_statistic
import batman
import emcee
import corner
import os, sys, time
import pandas as pd

In [0]:
data = pd.read_csv('wasp18_data.csv')

In [0]:
data

In [0]:
x = data.iloc[:,0].to_numpy()
y = data.iloc[:,1].to_numpy()

In [0]:
targ_name = 'WASP-43b'

# MCMC parameters
nsteps = 100000
burn_in = 10000
ndim = 1
nwalkers = 100

In [0]:
# Priors.
def lnprior(theta): 
	per = theta
	if (0. < per < 2) :
		return 0
	return -np.inf

In [0]:
def lnlike(theta, x, y, t0, sigma):
  per  = theta   
  model = t0 + x*per  
  inv_sigma2 = 1.0 / (sigma**2)
  return -0.5*(np.sum((y-model)**2*inv_sigma2))

# Define log of probability function.
def lnprob(theta, x, y, t0, sigma):
  lp = lnprior(theta)
  if not np.isfinite(lp):
    return -np.inf
  return lp + lnlike(theta, x, y, t0, sigma)

In [0]:
t0 = 2456728.74251
per_i = 0.813475
sigma = 0.000721

initial_params = per_i 

# Initialize walkers around maximum likelihood.
pos = [initial_params + 1e-5*np.random.randn(ndim) for i in range(nwalkers)]

# Set up sampler.
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(x, y, t0, sigma))

# Run MCMC for n steps and display progress bar.
width = 50
for i, result in enumerate(sampler.sample(pos, iterations=nsteps)):
    n = int((width+1) * float(i) / nsteps)
    sys.stdout.write("\r{}[{}{}]{}".format('sampling... ', '#' * n, ' ' * (width - n), ' (%s%%)' % str(100. * float(i) / nsteps)))
sys.stdout.write("\n")
print ('Sampling complete!')

sampling... [##################################################] (99.999%)
Sampling complete!


In [0]:
samples = sampler.flatchain

theta_max  = samples[np.argmax(sampler.flatlnprobability)]

In [0]:
theta_max

array([0.81383519])

# Fit decaying orbit model to detect how period changes with epoch number

In [0]:
targ_name = 'WASP-18b' 
 
# MCMC parameters
nsteps = 100000
burn_in = 20000
ndim = 2
nwalkers = 100

In [0]:
# Priors.
def lnprior(theta): 
	per, dpdt = theta
	if (0. < per < 2) and (0. > dpdt > -0.0005) :
		return 0
	return -np.inf

In [0]:
def lnlike(theta, x, y, t0, sigma):
  per, dpdt  = theta   
  model = t0 + x*per + 0.5*x*x*dpdt
  inv_sigma2 = 1.0 / (sigma**2)
  return -0.5*(np.sum((y-model)**2*inv_sigma2))

# Define log of probability function.
def lnprob(theta, x, y, t0, sigma):
  lp = lnprior(theta)
  if not np.isfinite(lp):
    return -np.inf
  return lp + lnlike(theta, x, y, t0, sigma)

In [0]:
# wasp-43
t0 = 2458366.69709000000
per_i = 0.94145299 
sigma = 0.000721
dpdt_i = 0

# wasp-19
#t0 = 2456021.70374
#per_i = 0.7888399
#dpdt_i = 0
#sigma = 0.00065836

initial_params = per_i, dpdt_i 

# Initialize walkers around maximum likelihood.
pos = [initial_params + 1e-5*np.random.randn(ndim) for i in range(nwalkers)]

# Set up sampler.
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(x, y, t0, sigma))

# Run MCMC for n steps and display progress bar.
width = 50
for i, result in enumerate(sampler.sample(pos, iterations=nsteps)):
    n = int((width+1) * float(i) / nsteps)
    sys.stdout.write("\r{}[{}{}]{}".format('sampling... ', '#' * n, ' ' * (width - n), ' (%s%%)' % str(100. * float(i) / nsteps)))
sys.stdout.write("\n")
print ('Sampling complete!')

samples = sampler.chain
 
# Discard burn-in. 
samples = samples[:, burn_in:, :].reshape((-1, ndim))

# Final params and uncertainties based on the 16th, 50th, and 84th percentiles of the samples in the marginalized distributions.
per_mcmc, dpdt_mcmc = map(
	lambda v: (v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(samples, [16, 50, 84], axis=0)))
    
print('Period: ' ,per_mcmc[0], per_mcmc[1])
print('dP/dt: ',  dpdt_mcmc[0],  dpdt_mcmc[1])

/usr/local/lib/python3.6/dist-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in double_scalars
  lnpdiff = f + nlp - state.log_prob[j]


sampling... [##################################################] (99.999%)
Sampling complete!
Period:  0.9414512381999057 1.313257127488754e-07
dP/dt:  -5.780839285423718e-10 4.191861467623709e-11


In [0]:
samples = sampler.flatchain

theta_max  = samples[np.argmax(sampler.flatlnprobability)]

In [0]:
theta_max

array([ 8.13835193e-01, -6.69906994e-17])

In [0]:
def model(theta, x=x, t0=t0):
  per, dpdt  = theta   
  model = t0 + x*per + 0.5*per*x*x*dpdt
  return model

In [0]:
def sample_walkers(nsamples,flattened_chain, y):
    models = []
    draw = np.floor(np.random.uniform(0,len(flattened_chain),size=nsamples)).astype(int)
    thetas = flattened_chain[draw]
    for i in thetas:
        mod = y-model(i)
        plt.plot(x, mod)
        plt.show()
        models.append(mod)
    spread = np.std(models,axis=0)
    med_model = np.median(models,axis=0)
    return med_model,spread
med_model, spread = sample_walkers(10, samples, y)

In [0]:
new_best_fit_model = model(theta_max)